# TP 12 Simulación - Call Center Metrogas

## Contexto
Se dispone de 3 tipos de puestos, y diferentes cantidades de cada uno de ellos:

- Emergencia (E)
- Comercial (C)
- Polifuncional (P)

Y dos colas:

- Cola para emergencia (NSE)
- Cola para comercial (NSC)

Los puestos comerciales y los puestos de emergencia solo atienden su propia cola. Los puestos polifuncionales atienden solo cuando todos los puestos comerciales y/o todos los puestos de emergencia estan ocupados y hay gente esperando en la cola para ser atendido,sera atendido por un polifuncional, dandole mas prioridad a los que estan en la cola de emergencia 

## Importando librerias de simpy

In [1]:
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections

_____

## Simulacion de una llegada

In [10]:
def distribucion_llegada(env,puestos,cont,n):
    while True:
    #for i in range(n):
    
        cont["NT"] += 1
        
        # Genero un cliente random 
        r = random.random()
        #print("Distribucion de generacion de tipo de cliente: %5.2f"%r)
        
        # Si genere un cliente para emergencia
        if r <= cont["PE"]:
            
            ce = Customer(name = "clienteE%02d"%cont["NT"])
            
            # Genero IA para el siguiente cliente y genero el timeout para su llegada 
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            print("T=%5.2f %s: COLA EMERGENCIA - llegue con un IA: %5.2f"%(env.now,ce.name,ia))
            colas(puestos) # Imprimo el estao de las colas
            
            # Genero la llamada
            env.process(ce.llamada_emergencia(env,puestos,cont))
            
        # Si genere un cliente para comercial    
        else: 
            
            cc = Customer(name = "clienteC%02d"%cont["NT"])
            # Genero IA para el siguiente cliente y genero el timeout para su llegada 
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            print("T=%5.2f %s: COLA COMERCIAL - llegue con un IA: %5.2f"%(env.now,cc.name,ia))
            colas(puestos) # Imprimo el estao de las colas
            
            # Genero la llamada
            env.process(cc.llamada_comercial(env,puestos,cont))

In [11]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(self,env,puestos,cont):
        
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        # puestos = [comercial, emergencia, polifuncional]
        emergencia, polifuncional = puestos[1].request(priority=0), puestos[2].request(priority=0)
        atendido = yield emergencia | polifuncional 
        #assert atendido == {emergencia}
        
        # Me atendieron por puestos polifuncionales?
        if polifuncional in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto polifuncional despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de polifuncional"%(env.now,self.name))
        
        # Me atendieron por un puesto de emergencia
        else:
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
            
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de emergencia"%(env.now,self.name))
            
            
    def llamada_comercial(self,env,puestos,cont):
        
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        # puestos = [comercial, emergencia, polifuncional]
        comercial, polifuncional = puestos[0].request(priority=0), puestos[2].request(priority=1) # comercial tn menos prioridad q emergencia
        atendido = yield comercial | polifuncional 
        
        # Me atendieron por puestos polifuncionales?
        if polifuncional in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA COMERCIAL - Me atendieron en un puesto polifuncional despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de polifuncional"%(env.now,self.name))
        
        # Me atendieron por un puesto comercial
        else:
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
            
            ta = generar_TA(cont)
            print("T=%5.2f, %s: COLA COMERCIAL - Me atendieron en un puesto comercial despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            print("T=%5.2f, %s: Sali de un puesto de comercial"%(env.now,self.name))

In [12]:
def colas(p):
    print ("Atendiendo:[c:%1d, e:%1d]"%(p[0].count,p[1].count))
    print ("En cola:[c:%1d, e:%1d]"%(len(p[0].queue),len(p[1].queue)))


In [13]:
def generar_TA(cont):
    #(a=5,b=615)
    a=5
    b=615
    ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
    
    while ta<a: 
        ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
        if ta>b:
            ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
            if ta>b:
                continue
            else:
                break
        
    while ta>b: 
        ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
        if ta<a:
            ta = abs(random.normalvariate(cont["mu"],cont["sigma"]))
            if ta<a:
                continue
            else:
                break
            
    return ta

Condiciones Iniciales

In [14]:
# Experimental data
NT = 0
maxTime = 14400 #segundos
maxClient = 10000
#random.seed(RANDOM_SEED)
PE = 0.12 #Porcentaje Emergencia
PC = 0.88 #Porcentaje Comercial
NEC = 0 #Cantidad de gente que espero en cola menor a 40 segundos
PEC = 0.0 #Porcentaje de espera en cola menor a 40 segundos

# TA - distribucion normal
mu = 267.64 
sigma = 297.40

# IA - uniforme
IAMin = 1
IAMax = 60

# Contadores
ci = {"multitasking":0,
     "mu":mu,
     "sigma":sigma,
     "min":IAMin,
     "max":IAMax,
     "PE":PE,
     "PC":PC,
     "emer":0,
     "com":0,
     "pol":0,
     "NEC":NEC,
     "NT":NT}

## Puestos de atencion - variables de control

!! Generar una version para automatizar esto, que genere diferentes combinaciones dado un rango

Y tmbn dejar la opcion de generarlo manualmente

In [19]:

print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
ci["com"] = cantCom
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())
ci["emer"] = cantEmer
print('Ingrese cantidad de puestos polifuncionales')
cantPolif = int(input())
ci["pol"] = cantPolif


Ingrese cantidad de puestos comerciales
60
Ingrese cantidad de puestos de emergencia
50
Ingrese cantidad de puestos polifuncionales
80


Modelo de Simulacion

In [20]:
env = simpy.Environment()
puestos = [simpy.PriorityResource(env,capacity=cantCom),
           simpy.PriorityResource(env,capacity=cantEmer),
           simpy.PriorityResource(env,capacity=cantPolif)] # menor el numero priority, mayor es la prioridad
           
env.process(distribucion_llegada(env,puestos,ci,n=maxClient))
env.run(until=maxTime)

T=49.22 clienteC932: COLA COMERCIAL - llegue con un IA: 49.22
Atendiendo:[c:0, e:0]
En cola:[c:0, e:0]
T=49.22, clienteC932: COLA COMERCIAL - Me atendieron en un puesto polifuncional despues de esperar  0.00
Atendiendo:[c:1, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC932: 57.16
T=69.24 clienteC933: COLA COMERCIAL - llegue con un IA: 20.02
Atendiendo:[c:1, e:0]
En cola:[c:0, e:0]
T=69.24, clienteC933: COLA COMERCIAL - Me atendieron en un puesto polifuncional despues de esperar  0.00
Atendiendo:[c:2, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC933: 88.18
T=89.44 clienteC934: COLA COMERCIAL - llegue con un IA: 20.21
Atendiendo:[c:2, e:0]
En cola:[c:0, e:0]
T=89.44, clienteC934: COLA COMERCIAL - Me atendieron en un puesto polifuncional despues de esperar  0.00
Atendiendo:[c:3, e:0]
En cola:[c:0, e:0]
Tiempo de Atencion generado para clienteC934: 362.06
T=106.38, clienteC932: Sali de un puesto de polifuncional
T=141.80 clienteC935: COLA COMERCIAL -

T=10896.68 clienteC1286: COLA COMERCIAL - llegue con un IA: 20.09
Atendiendo:[c:60, e:44]
En cola:[c:250, e:0]
T=10906.42 clienteC1287: COLA COMERCIAL - llegue con un IA:  9.74
Atendiendo:[c:60, e:44]
En cola:[c:251, e:0]
T=10944.33, clienteE1279: Sali de un puesto de emergencia
T=10949.68 clienteC1288: COLA COMERCIAL - llegue con un IA: 43.26
Atendiendo:[c:60, e:44]
En cola:[c:252, e:0]
T=11000.99 clienteC1289: COLA COMERCIAL - llegue con un IA: 51.31
Atendiendo:[c:60, e:44]
En cola:[c:253, e:0]
T=11010.72 clienteC1290: COLA COMERCIAL - llegue con un IA:  9.73
Atendiendo:[c:60, e:44]
En cola:[c:254, e:0]
T=11070.35 clienteE1291: COLA EMERGENCIA - llegue con un IA: 59.63
Atendiendo:[c:60, e:44]
En cola:[c:255, e:0]
T=11070.35, clienteE1291: COLA EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar  0.00
Atendiendo:[c:60, e:45]
En cola:[c:255, e:0]
Tiempo de Atencion generado para clienteE1291: 145.99
T=11072.02, clienteE1275: Sali de un puesto de emergencia
T=11103.

Results

In [21]:
print("===========================================================================")
print("RESULTADOS:")
print("Clientes totales: %1d"%ci["NT"])
print("Tiempo de espera en cola total: %1d"%ci["NEC"]) # Sacar esto despues
PEC = (ci["NEC"] / ci["NT"]) * 100
# Porcentaje de espera menor a 40 segundos
print("Porcentaje de espera en cola menor a 40 segundos: %5.2f"%PEC)
# Objetio: Mayor o igual al 90%

print('Cantidad de puestos comerciales',cantCom)
print('Cantidad de puestos de emergencia',cantEmer)
print('Cantidad de puestos polifuncionales',cantPolif)
print("Cantidad de gente atendido por polifuncional",ci["multitasking"])

RESULTADOS:
Clientes totales: 1402
Tiempo de espera en cola total: 216
Porcentaje de espera en cola menor a 40 segundos: 15.41
Cantidad de puestos comerciales 60
Cantidad de puestos de emergencia 50
Cantidad de puestos polifuncionales 80
Cantidad de gente atendido por polifuncional 140
